In [27]:
import pandas as pd
pd.options.mode.chained_assignment = None
import numpy as np
import scipy.sparse as sparse
import itertools
import os
import math
import openpyxl
import inspect
import xlsxwriter

In [28]:
# Python program to convert a list
# to string using join() function
    
# Function to convert  
def listToString(s): 
    
    # initialize an empty string
    str1 = " " 
    
    # return string  
    return (str1.join(s))

In [29]:
#b = [x for x in str_list if x != '']
def conList(str_list):
    str_list = str_list.split(' ')
    b = []
    for item in str_list:
        if item != '':
            if item.isdigit():
                b.append([int(item)])
            else:
                helplist = []
                start = int(item.split('to')[0])
                end = int(item.split('to')[1]) + 1
                for elem in range(start, end):
                    helplist.append(elem)
                b.append(helplist)                                
    flat_list = []
    for sublist in b:
        for item in sublist:
            flat_list.append(item)
    return flat_list

In [30]:
def printListtoRbt(listToRbt):
    listToPrint = ""
    for elem in listToRbt:
        listToPrint = listToPrint + " " +str(elem)
    return print(listToPrint)

In [31]:
def dfmaker(pathtocsv):
    df = pd.read_csv(os.getcwd() + '\\Data\\' + pathtocsv + '.csv', encoding = "utf-16", na_values='-', sep=";", decimal=",", low_memory=False).fillna(0)
    if 'bar' in pathtocsv.lower():
        barsdf = df.drop(df.index[len(df)-1])
        barsdf = barsdf.astype({"Bar": int, "Node 1": int, "Node 2": int})        
        return barsdf
    elif 'node' in pathtocsv.lower():
        nodesdf = df.drop(df.index[len(df)-1]).iloc[:, :-1].drop_duplicates(subset="Node").apply(pd.to_numeric)
        #forcesdf = pd.read_csv('Forces.csv', encoding = "utf-16", sep=';')
        return nodesdf
    elif 'force' in pathtocsv.lower():
        forcesdf = df.join(df['Bar/Node/Case'].str.split('/', expand=True).rename(columns={0:'Bar', 1:'Node', 2:'Case'})).iloc[:, 1:].apply(pd.to_numeric)
        forcesdf = forcesdf.astype({"Bar": int, "Node": int, "Case": int})
        return forcesdf
    else:
        return ValueError   
        

In [32]:
def barsdfmod(barsdf, nodesdf):
    #Modification of barsdf, add "direction"
    barsdf = barsdf.merge(nodesdf, left_on='Node 1', right_on='Node')
    barsdf = barsdf.merge(nodesdf, left_on='Node 2', right_on='Node')
    barsdf['deltaX'] = barsdf['X (m)_x'] - barsdf['X (m)_y']
    barsdf['deltaY'] = barsdf['Y (m)_x'] - barsdf['Y (m)_y']
    #barsdf['deltaZ'] = barsdf['Z (m)_x']-barsdf['Z (m)_y']
    #barsdf['signX'] = np.sign(barsdf['deltaX'])
    #barsdf['signY'] = np.sign(barsdf['deltaY'])
    barsdf['Direction'] = barsdf.apply(lambda r: tuple(r)[-2:], axis=1).apply(np.array)
    #barsdf['Angle'] = np.degrees(np.arctan(barsdf['deltaX'] / barsdf['deltaY']))
    barsdf = barsdf[['Bar', 'Node 1', 'Node 2', 'Direction', 'Section']]
    return barsdf

In [33]:
def extremum(listofelem, maxminabs):
    if maxminabs == 'max':
        return max(listofelem)
    elif maxminabs == 'min':
        return min(listofelem)
    else:
        return abs(listofelem)

In [34]:
def nodeinverse(barsdf, Node):
    df = pd.concat([barsdf[barsdf['Node 1'] == Node], barsdf[barsdf['Node 2'] == Node]])
    Node2 = []
    for i in range(len(df)):
        if df['Node 1'].tolist()[i]!=Node:
            Node2.append(df['Node 1'].tolist()[i])
            df['Direction'].iloc[i] = -df['Direction'].iloc[i]
            #df['Angle'].iloc[i] = -df['Angle'].iloc[i]
        else:
            Node2.append(df['Node 2'].tolist()[i])
    Node1 = [Node for i in range(len(df))]
    df['Node 1'] = Node1
    df['Node 2'] = Node2
    return df

In [35]:
def extremalforces(barsdf, forcesdf, listofnodes, forcetype, maxminabs, section=True):
    
    forcestypedict = {
      "FX": "FX (kN)",
      "FY": "FY (kN)",
      "FZ": "FZ (kN)",
      "MX": "MX (kNm)",
      "MY": "MY (kNm)",
      "MZ": "MZ (kNm)"
    }   

    Forcemin = True
    Forcemax = True

    #zmiana kolejności Node'ow
    for Node in listofnodes:
        df = nodeinverse(barsdf, Node)
        selforcesdf = forcesdf[forcesdf['Bar'].isin(df['Bar'].tolist())]
        selforcesdf = selforcesdf[selforcesdf['Node'] == Node]
        df = df.merge(selforcesdf, on='Bar', how='right')
        df.drop('Node', inplace=True, axis=1)
        if type(section) is not str:
            selcolumn = df[forcestypedict[forcetype]]
        else:
            if section in list(df['Section']):
                selcolumn = df[df['Section'] == section][forcestypedict[forcetype]]
            else:
                selcolumn = math.sqrt(-1) #ErrorValue
        
        if Forcemin:
            Forcemin = selcolumn.min()
        if Forcemax:
            Forcemin = selcolumn.max()
        
        if maxminabs == 'min':
            if (selcolumn.min() <= Forcemin):
                dfout = df
                idx = selcolumn.idxmin()
                Forcemin = selcolumn.min()
                dfout = dfout[dfout['Case'] == dfout.loc[idx]['Case']]
        elif maxminabs == 'max':
            if (selcolumn.max() >= Forcemax):
                dfout = df
                Forcemax = selcolumn.max()
                idx = selcolumn.idxmax()
                dfout = dfout[dfout['Case'] == dfout.loc[idx]['Case']]
        elif maxminabs == 'abs':
            if (abs(selcolumn).max() >= Forcemax):
                dfout = df
                idx = abs(selcolumn).idxmax()
                Forcemax = abs(selcolumn).max()
                dfout = dfout[dfout['Case'] == dfout.loc[idx]['Case']]
    return dfout

In [36]:
def extremalforcesall(inputdict, args):
    try:
        forcetype, maxminabs, section = args
    except:
        forcetype, maxminabs = args
        section = True
    
    forcestypedict = {
      "FX": "FX (kN)",
      "FY": "FY (kN)",
      "FZ": "FZ (kN)",
      "MX": "MX (kNm)",
      "MY": "MY (kNm)",
      "MZ": "MZ (kNm)"
    }  
    maxvalue = 0
    #dfout = math.sqrt(-1) ErrorValue
    for key in inputdict:
        try:
            df = extremalforces(inputdict[key][1][0], inputdict[key][1][2], inputdict[key][0], forcetype, maxminabs, section)         
            if type(section) is not str:
                maxcurforce = abs(df[forcestypedict[forcetype]]).max()
            else:
                if section in list(df['Section']):
                    maxcurforce = abs(df[df['Section'] == section][forcestypedict[forcetype]]).max()
                else:
                    maxcurforce = math.sqrt(-1) #ErrorValue

            if maxcurforce > maxvalue: 
                maxvalue = maxcurforce
                dfout = df
        except:
            None
    return dfout    

In [37]:
# conno = 'CON5'
# conList_p1 = ' 21 22 27 33 45 54 59  '
# conList_p2 = ' 1030 1049 1066'
# conList_p3 = ' '

In [99]:
# conno = 'CON7'
# conList_p1 = ' '
# conList_p2 = ' 1071 1080 1082 1088 '
# conList_p3 = ' 1384 1463 1478 1508 '
conno = 'CON7'
conList_p1 = ' '
conList_p2 = ' 1054 '
conList_p3 = ' 1449 1463 '

In [75]:
# conno = 'CON8'
# conList_p1 = ' '
# conList_p2 = ' 1054 '
# conList_p3 = '  '

In [100]:
conList_p1 = conList(conList_p1)
conList_p2 = conList(conList_p2)
conList_p3 = conList(conList_p3)

inputdict = {
    'p1' : [conList_p1, [dfmaker('Barsp1'), dfmaker('Nodesp1'), dfmaker('Forcesp1')]],
    'p2' : [conList_p2, [dfmaker('Barsp2'), dfmaker('Nodesp2'), dfmaker('Forcesp2')]],
    'p3' : [conList_p3, [dfmaker('Barsp3'), dfmaker('Nodesp3'), dfmaker('Forcesp3')]]
}
for key in inputdict:
    inputdict[key][1][0] = barsdfmod(inputdict[key][1][0], inputdict[key][1][1])

In [101]:
listofcases = [['FZ', 'abs'],['FZ', 'abs', 'IPE 160'],['MY', 'abs'],['FX', 'min', 'PO 16']]

In [102]:
for elem in listofcases:
    display(extremalforcesall(inputdict, elem))

,Bar,Node 1,Node 2,Direction,Section,FX (kN),FY (kN),FZ (kN),MX (kNm),MY (kNm),MZ (kNm),Case
1052,168,1449,1450,"[-0.0, 2.0]",RP 250x150x6,0.05,-0.01,-45.17,-0.01,-76.69,0.03,1067
2337,169,1449,1448,"[0.0, -2.0]",RP 250x150x6,0.23,0.02,38.62,-0.06,-76.70,0.05,1067
3622,301,1449,1463,"[-4.0, 0.0]",IPE 160,-0.49,-0.00,11.51,-0.00,0.00,0.00,1067
4907,424,1449,1434,"[4.0, -0.0]",IPE 160,-0.24,-0.00,-11.51,0.00,0.00,0.00,1067
6192,731,1449,1678,"[-0.0, -0.0]",RO 127x16,106.86,0.45,0.00,0.02,-0.05,0.00,1067
7477,1040,1449,1577,"[2.0, 2.0]",PO 16,-0.40,0.00,0.00,0.00,0.00,0.00,1067
8762,1046,1449,1578,"[-2.0, 2.0]",PO 16,0.00,0.00,0.00,0.00,0.00,0.00,1067


,Bar,Node 1,Node 2,Direction,Section,FX (kN),FY (kN),FZ (kN),MX (kNm),MY (kNm),MZ (kNm),Case
1049,184,1463,1464,"[-0.0, 2.0]",RP 250x150x6,0.10,0.00,-43.66,-0.00,-65.45,0.01,1064
2334,185,1463,1462,"[0.0, -2.0]",RP 250x150x6,0.40,-0.01,31.89,-0.01,-65.45,0.02,1064
3619,301,1463,1449,"[4.0, -0.0]",IPE 160,-0.48,-0.00,-11.51,-0.00,0.00,0.00,1064
4904,315,1463,1478,"[-4.0, 0.0]",IPE 160,-0.25,0.00,11.51,0.00,0.00,0.00,1064
6189,732,1463,1679,"[-0.0, -0.0]",RO 127x16,98.65,0.07,0.00,0.01,-0.01,-0.00,1064
7474,1014,1463,1570,"[-2.0, -2.0]",PO 16,-0.44,0.00,0.00,0.00,0.00,0.00,1064
8759,1021,1463,1571,"[-2.0, 2.0]",PO 16,0.00,0.00,0.00,0.00,0.00,0.00,1064
10044,1044,1463,1578,"[2.0, 2.0]",PO 16,-0.10,0.00,0.00,0.00,0.00,0.00,1064


,Bar,Node 1,Node 2,Direction,Section,FX (kN),FY (kN),FZ (kN),MX (kNm),MY (kNm),MZ (kNm),Case
1052,94,1054,1055,"[-0.0, 2.0]",RP 250x150x4,0.38,0.06,-30.42,0.03,-51.43,-0.05,1067
2337,95,1054,1053,"[0.0, -1.9999999999999991]",RP 250x150x6,0.04,-0.05,41.74,-0.02,-78.84,-0.05,1067
3622,380,1054,1350,"[3.9990000000000023, -0.0009999999999994458]",IPE 160,-2.77,-0.00,-10.12,-0.01,0.00,0.00,1067
4907,397,1054,1071,"[-4.0, 0.0]",IPE 160,0.24,0.00,9.67,0.00,0.00,0.00,1067
6192,742,1054,1332,"[0.0, 0.0]",RO 177.8x16,92.00,0.00,-3.47,0.00,0.05,-27.42,1067
7477,1079,1054,1175,"[2.0189999999999984, 2.0]",PO 16,-0.49,0.00,0.00,0.00,0.00,0.00,1067
8762,1082,1054,1177,"[-2.0, 2.0]",PO 16,0.00,0.00,0.00,0.00,0.00,0.00,1067


,Bar,Node 1,Node 2,Direction,Section,FX (kN),FY (kN),FZ (kN),MX (kNm),MY (kNm),MZ (kNm),Case
810,94,1054,1055,"[-0.0, 2.0]",RP 250x150x4,8.62,-0.50,-21.00,-0.05,-31.26,0.79,825
2095,95,1054,1053,"[0.0, -1.9999999999999991]",RP 250x150x6,-2.86,0.19,14.31,0.04,-27.71,0.79,825
3380,380,1054,1350,"[3.9990000000000023, -0.0009999999999994458]",IPE 160,-11.37,0.60,-5.16,0.00,-0.00,-0.00,825
4665,397,1054,1071,"[-4.0, 0.0]",IPE 160,-4.21,-0.60,4.94,-0.00,-0.00,-0.00,825
5950,742,1054,1332,"[0.0, 0.0]",RO 177.8x16,45.46,0.00,6.20,0.00,-0.09,3.55,825
7235,1079,1054,1175,"[2.0189999999999984, 2.0]",PO 16,0.00,0.00,0.00,0.00,0.00,0.00,825
8520,1082,1054,1177,"[-2.0, 2.0]",PO 16,-17.93,0.00,0.00,0.00,0.00,0.00,825


In [103]:
with pd.ExcelWriter(conno + '.xlsx', engine='xlsxwriter') as writer:
    for elem in listofcases:
        df = extremalforcesall(inputdict, elem)
        df.to_excel(writer, sheet_name = listToString(elem))